In [ ]:
"""
To check the progress of the neuron decomposition

"""

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

import minfig
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
#du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-26 10:09:38,466 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-26 10:09:38,467 - settings - Setting database.user to celiib
INFO - 2020-11-26 10:09:38,468 - settings - Setting database.password to newceliipass
INFO - 2020-11-26 10:09:38,478 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-26 10:09:38,479 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-26 10:09:38,493 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


# Inspecting a certain decomposition

In [75]:
import neuron_visualizations as nviz
curr_seg_id = 864691134884742394

In [76]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=curr_seg_id)).fetch("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [77]:
nviz.visualize_neuron(neuron_obj[0],
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton
Working on  new stand alone scatter points


In [15]:
nviz.plot_spines(neuron_obj[0])


 Working on visualization type: mesh
Working on  new stand alone scatter points



 Working on visualization type: mesh

 Working on visualization type: skeleton
Working on  new stand alone scatter points


# The restart of the Decomposition

In [ ]:
(schema.jobs & "table_name='__decomposition'" & "status='reserved'" & "timestamp>'2020-11-24 23:00:00'")#.delete()

In [ ]:
len(minnie.Decomposition())

In [78]:
(schema.jobs & "table_name='__decomposition'" & "status='reserved'")#.delete()# & "error_message='Exception: The output from mesh_correspondence_adaptive_distance was nothing: curr_branch_face_correspondence'").fetch("key")

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,00a88793585592795df96b9fa5ab9d84,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.161,at-node34,30,108530,2020-11-25 19:33:04
__decomposition,00ea0c8ad9a653ff707c0c1ab836e4d2,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,29,108605,2020-11-25 18:22:49
__decomposition,01727da8d33d9248c3bd4720c8e069df,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,108351,2020-11-25 17:11:30
__decomposition,01a2d213314f80ad2728ffd6a215b536,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.175,at-node48,30,108145,2020-11-25 19:22:49
__decomposition,02218f1cdb5713f0d7b152bc4759406c,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.64,jr-compute001,29,108484,2020-11-25 17:32:02
__decomposition,02594eb7eb27e2cd758e0166ae482e56,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.160,at-node33,29,108520,2020-11-25 19:26:51
__decomposition,0269fdd89c6bc63b3e491f862a76ddff,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.81,at-compute003,29,108225,2020-11-25 19:11:52
__decomposition,02a78cc9043487fab92565e7b419f643,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.128,at-node1,29,108136,2020-11-25 19:39:58
__decomposition,0479bf481d5e42935400d2aaf82a2fc9,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,30,108102,2020-11-25 16:23:38
__decomposition,04d1fe0b48179aa200fb6ce6a6a2a2af,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,108428,2020-11-25 17:26:10


In [93]:
minnie.Decomposition()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),decomposition,n_vertices number of vertices,n_faces number of faces,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134026921572,0,0.25,=BLOB=,10752,22449,0,0,0,1,1,1,1,9257.060907158155,9257.060907158155,9257.061,219.966,219.966,219.966,219.966,11,0.0011882821243505153,11.0,9257.061,1,nan,nan,5909217334.438089,9946671.094996769,638347.0297649984,638347.0233628243,5909217334.438089,38.5579
864691134309610836,0,0.25,=BLOB=,14274,31974,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,24.5438
864691134412252598,0,0.25,=BLOB=,13529,29348,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,23.9303
864691134413248927,0,0.25,=BLOB=,14211,31270,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,25.1809
864691134413821752,0,0.25,=BLOB=,11696,25792,1,1,0,1,1,3,3,9299.020064879269,9299.020064879269,1599.272,34.572,34.572,103.183,103.183,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,125.4067
864691134431433407,0,0.25,=BLOB=,11510,24714,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,20.9624
864691134433153501,0,0.25,=BLOB=,11585,23209,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,22.4911
864691134466251864,0,0.25,=BLOB=,10808,23322,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,24.6378
864691134467173584,0,0.25,=BLOB=,14804,31831,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,20.0085
864691134488345090,0,0.25,=BLOB=,12908,27823,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,nan,nan,0.0,0.0,0.0,0.0,0.0,8.8056


In [14]:
len(minnie.Decomposition() & "n_somas = 1" & "n_faces>500000") - 10215

11398

In [ ]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

In [ ]:
minnie.BaylorSegmentCentroid()

In [52]:
minnie.SegToDecimateFromNuclei() & "segment_id=864691136105493209"

segment_id segment id for those to be decimated
864691136105493209


# Inspecting BaylorSegmentCentroid

In [ ]:
# The final number should be 87438
len(minnie.BaylorSegmentCentroid())

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000

key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei())
len(key_source)
len(key_source & minnie.BaylorSegmentCentroid.proj())

In [ ]:
86836

In [ ]:
"""
87438 is the goal: 85993
"""
87438-85993

In [ ]:
len(minnie.Decomposition())

In [ ]:
minnie.Decimation() & (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj()

In [ ]:
73874

In [ ]:
minnie.SegToDecimateFromNuclei()

In [ ]:
#(schema.jobs & "table_name='__baylor_segment_centroid'" & "status='reserved'").delete()

In [ ]:
len(minnie.Decomposition())-9385

In [ ]:
minnie.Decomposition() & "n_somas=1" & "n_vertices > 40000"

In [ ]:
neuron_obj = (minnie.Decomposition() & "segment_id=864691134884743418").fetch1("decomposition")

In [ ]:
neuron = reload(neuron)
tu = reload(tu)
nru = reload(nru)

In [ ]:
neuron_obj.calculate_spines(query='median_mesh_center > 150 and n_faces_branch>100')

In [ ]:
nviz.plot_spines(neuron_obj)

In [ ]:
neuron_obj[0][3].spines_volume#["median_mesh_center"]

In [ ]:
ret_col = nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh","skeleton"],
                      limb_branch_dict=dict(L3="all"),
                     return_color_dict=True)

In [ ]:
neuron_obj[0][7].width_array["median_mesh_center"]

In [ ]:
neuron_obj[0][7].mesh.show()

In [ ]:
nviz.plot_limb_concept_network_2D(neuron_obj,node_colors=ret_col)

In [ ]:
nviz.plot_spines(neuron_obj)

In [ ]:
ret_col = nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh","skeleton"],
                      mesh_resolution="limb",
                      skeleton_resolution="limb",
                      limb_branch_dict="all",
                               return_color_dict=True)

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj,
                                   node_colors=ret_col)

# Checking the Decomposition Table

In [ ]:
import neuron_visualizations as nviz
nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh","skeleton"],
                      limb_branch_dict="all")

In [ ]:
(schema.jobs & "table_name='__decomposition'" & "status='error'" & "key_hash='100cc5b6c8387409cc7c30e9ae956955'").delete()#.fetch1("key")

In [ ]:
dj.config["display.limit"] = 10
(schema.jobs & "table_name='__decomposition'" & "status='error'")#.fetch1("key")#.delete()#.fetch1("key")

In [ ]:
(schema.jobs & "table_name='__decomposition'" & "status='error'")#& "key_hash='1cfdce50b5902366c8c6cedc200f6840'").fetch1("error_stack")

In [ ]:
ret_key = (schema.jobs & "table_name='__decomposition'" & "status='error'"& "key_hash='79a9341cc14a9158d4dcc011a997f814'").fetch1("key")
ret_key

In [ ]:
from datajoint_utils import *
schema.external['decomposition'].delete(delete_external_files=True)

In [ ]:
decomp_path = Path("/mnt/dj-stor01/platinum/minnie65/02/decomposition/")
for f in decomp_path.iterdir():
    print(f)

In [ ]:
20*70000/60/24/300

In [ ]:
all_run_times = minnie.Decomposition.fetch("run_time")
np.median(all_run_times)

In [ ]:
(minnie.schema.jobs & "table_name='__decomposition'" & "status='reserved'" & 'timestamp<"2020-11-22 19:00:00"').delete()

In [ ]:
(minnie.schema.jobs & "table_name='__decomposition'" & "status='reserved'")

In [ ]:
len(minnie.Decomposition())-846

In [ ]:
neuron_obj = (minnie.Decomposition() & "segment_id=864691136903062706").fetch1("decomposition")

In [ ]:
import neuron_visualizations as nviz
ret_col = nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh","skeleton"],
                      #mesh_resolution="limb",
                      #skeleton_resolution="limb",
                      limb_branch_dict="all",
                     return_color_dict=True)
                  

In [ ]:
minnie.decompo

In [ ]:
decimation_version = 0
decimation_ratio = 0.25

key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroidExternal() & "multiplicity>0").proj())
key_source

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj,node_colors=ret_col)

In [ ]:
soma_mesh = main_mesh=neuron_obj["S0"].mesh

In [ ]:
import neuron
neuron = reload(neuron)
neuron_obj = neuron.Neuron(neuron_obj)

In [ ]:
tu = reload(tu)

In [ ]:
nviz.plot_objects(main_mesh=neuron_obj["S0"].mesh,
                meshes=neuron_obj.non_soma_touching_meshes,
                  meshes_colors="random")

In [ ]:
ret_col = nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh","skeleton"],
                      #mesh_resolution="limb",
                      #skeleton_resolution="limb",
                        mesh_soma=False,
                      limb_branch_dict=dict(L6="all"))
                  

In [ ]:
nviz.plot_spines(neuron_obj)

# Deleteing A lot of the Tables

In [ ]:
"""
Pseudocode: 
Tables that need to be dropped:
-


"""

minnie.BaylorSegmentCentroidTest.drop()

In [ ]:
soma_path = Path("/mnt/dj-stor01/platinum/minnie65/02/somas/")

In [ ]:
for j,f in enumerate(soma_path.iterdir()):
    if j > 100:
        break
    print(f)

In [ ]:
from datajoint_utils import *
schema.external['somas'].delete(delete_external_files=True)